## Script to implement design and optimization model
### Author: Dharik S. Mallapragada
##### Input files:
##### a) PV resource file (5796_23.65_68.75_tmy.csv)
##### b) Cost parameter, defined for each scenario as a separate folder under the folder "CostScenarios"
##### Note: Script Gurobi license to run sucessfully.

In [ ]:
import pandas as pd
import numpy as np
import os
import pyomo.environ as en
from pyomo.opt import SolverFactory
from Electrolyzer_v11 import build_model, GetStaticOutputs
import pickle
import gzip
import copy

# Define Gurobi parameters

In [ ]:
opt = SolverFactory("gurobi")
opt.options['MIPGap'] = 0.01
opt.options['TimeLimit'] = 60*60*4
#opt.options['Threads'] = 4

## Define functions used in analysis

In [ ]:
def read_PV_avail_df(pv_avail_filename):
    return pd.read_csv(pv_avail_filename, 
                       index_col=0, parse_dates=True,
                       header=None, squeeze=True)

def model_variables_to_dict(model):
    solution = {}
    for variable in model.component_objects(en.Var, active=True):
        if len(variable) == 1 and None in variable:
            solution[variable.name] = en.value(variable)
        else:
            solution[variable.name] = {index: en.value(variable[index]) for index in variable.index_set()}
    return solution

def load_variables(model, model_variables):
    for variable_name, variable_values in model_variables.items():
        if isinstance(variable_values, dict):
            for vindex, index_value in variable_values.items():
                getattr(model, variable_name)[vindex].value = index_value
        else:
            getattr(model, variable_name).value = variable_values

def load_or_solve(pickle_filename, model, warmstart=False):
    # loads the model from the file if it exists
    # solves if not
    # returns pickle dict
    if os.path.exists(pickle_filename):
        with gzip.open(pickle_filename, 'rb') as f:
            pickle_dict = pickle.load(f)
        model_variables = {k: v for k, v in pickle_dict.items() if k.startswith('v')}
        load_variables(model, model_variables)
    else:
        results = opt.solve(model, report_timing=True, tee=True, warmstart=warmstart)
#         log_infeasible_constraints(model)

        pickle_dict = model_variables_to_dict(RefModel)
        model_variables = copy.copy(pickle_dict)

        pickle_dict['LowerBound'] = results.problem[0]['Lower bound']
        pickle_dict['UpperBound'] = results.problem[0]['Upper bound']
        pickle_dict['TerminationCondition'] = str(results.solver[0]['Termination condition'])
        pickle_dict['Time'] = results.solver[0]['Time']
        pickle_dict['WallTime'] = results.solver[0]['Wall time']

    return (pickle_dict, model_variables)

def write_pickle(pickle_filename, pickle_dict):
    if not os.path.exists(pickle_filename):
        with gzip.open(pickle_filename, 'wb') as f:
            pickle.dump(pickle_dict, f)


def add_solve_info(summary, pickle_dict):
    summary['LowerBound'] = pickle_dict['LowerBound']
    summary['UpperBound'] = pickle_dict['UpperBound']
    summary['TerminationCondition'] = pickle_dict['TerminationCondition']
    summary['Time'] = pickle_dict['Time']
    summary['WallTime'] = pickle_dict['WallTime']

## Run model for a particular scenario and store outputs

In [ ]:
Outputs = []
for cost_scenario in ['2020_AG']:
    cost_scenario_folder = os.path.join('CostScenarios', cost_scenario)
    
    StorageData = pd.read_excel(os.path.join(cost_scenario_folder, 'StorageData.xlsx'),index_col=[0])
    Discount_rate = 0.054  # Discount rate %
    Lifetime = 20.0  # Lifetime in years
    # Capital charge factor to annualize investment costs
    CCF_val = 1/float((Discount_rate+1)/float(Discount_rate)*(1-1/(1+Discount_rate)**Lifetime))  
    PVData = pd.read_excel(os.path.join(cost_scenario_folder, 'PVData.xlsx'),'Data',index_col=[0]) # PV cost data
    ElyData = pd.read_excel(os.path.join(cost_scenario_folder, 'ElyData.xlsx'),'Data',index_col=[0]) # Electrolyzer cost data
    H2StData = pd.read_excel(os.path.join(cost_scenario_folder, 'H2StData.xlsx'),'Data',index_col=[0]) # H2 storage cost data
    cf_file = '5796_23.65_68.75_tmy.csv' # PV resource availability defined for a single location
    PVAvail_tmy = read_PV_avail_df(cf_file)
    productionCommitmentLB = int(np.floor(len(PVAvail_tmy) * .95)) # Minimum requirement for annual plant availability =95%
    minimumProductionShutdownLength = 12 # Minimum number of hours system has to be turned down
    P_Electricity = 120.0 #$/MWh # Price of exported grid electricity
    LMPData = pd.Series(P_Electricity, index=range(len(PVAvail_tmy))) # 8760 x 1 vector of electricity prices 

    RefModel = build_model(PVAvail_tmy.values, LMPData.values, PVData,
                           StorageData, ElyData, H2StData, CCF_val,
                           productionCommitmentLB=productionCommitmentLB,
                           minimumProductionShutdownLength=minimumProductionShutdownLength) # Construct pyomo model

    for r in range(1,len(PVAvail_tmy.values)+1):
        RefModel.vACPowtoGrid[r].fix(0)
        RefModel.vH2PlantOutputSlack[r].fix(0)# Fix the grid exports (MW) =0

    pickle_filename = f'{cost_scenario}_h2.p.gz'
    (pickle_dict, _) = load_or_solve(pickle_filename, RefModel) # Function to solve model and store values of decision variables

    Output = GetStaticOutputs(RefModel) # Store key metrics of interest in a dict along with other run information
    Output['CostScenario'] = cost_scenario
    Output['SolarFilename'] = cf_file
    Output['ProductionCommitmentLB'] = productionCommitmentLB
    Output['MinimumProductionShutdownLength'] = minimumProductionShutdownLength
    Output['P_Electricity'] = 0
    add_solve_info(Output, pickle_dict)

    Outputs.append(Output)
    pd.DataFrame(Outputs).to_csv(f'summary.csv')

    write_pickle(pickle_filename, pickle_dict) # store value of decision variables as a pickle

    # Grid Sales- Resolving the scheduling problem with grid exports allowed - for this run we fix size of various equipment
    for r in range(1,len(PVAvail_tmy.values)+1):
        RefModel.vACPowtoGrid[r].unfix()

    # Fix all the variables corresponding to equipment sizing (except inverter size which is allowed to vary)    
    for variable in ['vPVInstalledMW', 'vElyInstalledMW', 'vCompInstalledMW', 'vH2StInstalledNumber']:
        getattr(RefModel, variable).fix()
    RefModel.vStInstalledMW['es1'].fix()

    pickle_filename = f'{cost_scenario}_cogen.p.gz'
    (pickle_dict, _) = load_or_solve(pickle_filename, RefModel, warmstart=True) # Solve the model
    
       # Store key metrics of interest in a dict along with other run information
    Output = GetStaticOutputs(RefModel)
    Output['CostScenario'] = cost_scenario
    Output['SolarFilename'] = cf_file
    Output['ProductionCommitmentLB'] = productionCommitmentLB
    Output['MinimumProductionShutdownLength'] = minimumProductionShutdownLength
    Output['P_Electricity'] = P_Electricity
    add_solve_info(Output, pickle_dict)

    Outputs.append(Output)
    pd.DataFrame(Outputs).to_csv(f'summary.csv')

    write_pickle(pickle_filename, pickle_dict) # store value of decision variables as a pickle